# Run from jupyter

This page give an overview of the details of running a jupyter kernel from the jupyter notebook. It doesn't have much practical sence but extremely usefull for purposes of this project.

In [ ]:
import sys
import importlib
import multiprocessing
from ipykernel.kernelbase import Kernel
from ipykernel.kernelapp import IPKernelApp
from jupyter_client.blocking import BlockingKernelClient

sys.path.append("/tmp")

## Create new instance

The main problem associated with running kernel from jupyter environment - is that in execution environment already created kernel object. It follows singleton pattern which means that it has one instance over whole program, so if you try to create a new kernel you'll just get the same object - not create a new one. This can be overcome by running the kernel in the new process.

**Note** The process must be started in `spawn` or `forkserver` context so that it not to share resources with the parent process. For more details on this issue, see description on the different startup methods in the `multiprocessing` package. 

---

The following cell shows the instance of the `Kernel` class.

In [11]:
Kernel.instance()

As the result we got reference to the `IPythonKernel` - this is the kernel of the environment this notebook runned in.

So if you directly call `IPKernelApp.launch_instance` directly, you will just get error related to the fact that resources the kernel is trying to use are occupied, as shown in the next cell.

In [7]:
try:
    IPKernelApp.launch_instance()
except Exception as e:
    print(e)

init_sockets cannot be called twice!


The next code creates prints `Kernel.instance()` from the separate process.

In [15]:
%%writefile /tmp/show_kernel_instance.py
from ipykernel.kernelbase import Kernel

def process_target():
    print(Kernel.instance())

Overwriting /tmp/show_kernel_instance.py


In [12]:
from show_kernel_instance import process_target
context = multiprocessing.get_context("spawn")
p = context.Process(target=process_target)
p.start()

In the separate process, `Kernel.instance` refers to the different object.

## CLI arguments

Another issue related to starting the kernel in the jupyter notebook is that it can use command line arguments used by the parent process. Specifically, if a connection file was specified when setting up the kernel's new `IPKernelApp` object, these parameters will be inherited by the child process, and `IPKernelApp` will read and use these parameter.

- The most obvious solution is to simply skip the parameters of the parent process `sys.argv = ['python3']`.
- In some cases it's more beneficial to pass parameters directly as `list[str]` to `IPKernelApp.launch_instance` which will be used later in `IPKernelApp.initialize`.

---

**Note** The following example was run in vscode jupyter notebook implementation and in different implementation result may be different.

Two subsequent cells start a new process that prints `sys.argv` and calls the `IPKernelApp.parse_command_line` method which is an important part of the `IPKernelApp.launch_instance` instance. Dsiplays the `connection_file` attribute of the `IPKernelApp`.

In [40]:
%%writefile /tmp/show_arv.py
import sys
from ipykernel.kernelapp import IPKernelApp

def process_target():
    print(sys.argv)
    app = IPKernelApp.instance()
    app.parse_command_line()
    print("connection file:", app.connection_file)

Overwriting /tmp/show_arv.py


In [41]:
import show_arv
context = multiprocessing.get_context("spawn")
p = context.Process(target=show_arv.process_target)
p.start()

['/home/fedor/.virtualenvs/python/lib/python3.12/site-packages/ipykernel_launcher.py', '--f=/run/user/1000/jupyter/runtime/kernel-v3230a012ea78203d37cfc76381637f9f4dec70282.json']
connection file: /run/user/1000/jupyter/runtime/kernel-v3230a012ea78203d37cfc76381637f9f4dec70282.json


As a result, `IPKernelApp` will use the same connection file as the main kernel.

The following cell shows the result of `parsing_command_line` after resetting of the command line arguments from the parent process. 

In [42]:
%%writefile /tmp/show_arv.py
import sys
from ipykernel.kernelapp import IPKernelApp

def process_target():
    sys.argv = ["python3"]
    app = IPKernelApp.instance()
    app.parse_command_line()
    print("connection file:", app.connection_file)

Overwriting /tmp/show_arv.py


In [43]:
importlib.reload(show_arv)
context = multiprocessing.get_context("spawn")
p = context.Process(target=show_arv.process_target)
p.start()

connection file: 


As a result, there is no connection file in the child process.

An alternative solution is to pass the arguments as to the system as `List[str]`. The following cell runs `parse_command_line` that resets the `connection_file`.

In [44]:
%%writefile /tmp/show_arv.py
import sys
from ipykernel.kernelapp import IPKernelApp

def process_target():
    app = IPKernelApp.instance()
    app.parse_command_line(["--f", "value"])
    print("connection file:", app.connection_file)

Overwriting /tmp/show_arv.py


In [45]:
importlib.reload(show_arv)
context = multiprocessing.get_context("spawn")
p = context.Process(target=show_arv.process_target)
p.start()

connection file: value


As a result, the `connection_file` attribute of the `IPKernelApp` takes the values specified in the command line argument list.

## Implementation

This section describes the minimal working setup of the code that runs the kernel inside jupyter notebook.

---

The following cell defines the target for the process. It is only the `IPKernelApp.launch_instance` that redefines the connection file.

In [48]:
%%writefile /tmp/run_kernel.py
from ipykernel.kernelapp import IPKernelApp

def process_target():
    IPKernelApp.launch_instance(["-f", "/tmp/example_connection.json"])

Overwriting /tmp/run_kernel.py


The following code runs this kernel in a separate process.

In [50]:
from run_kernel import process_target
context = multiprocessing.get_context("spawn")
p = context.Process(target=process_target)
p.start()

NOTE: When using the `ipython kernel` entry point, Ctrl-C will not work.

To exit, you will have to explicitly quit this process, by either sending
"quit" from a client, or using Ctrl-\ in UNIX-like environments.

To read more about this, see https://github.com/ipython/ipython/issues/2049


To connect another client to this kernel, use:
    --existing /tmp/example_connection.json


To make sure that everything is working properly, the following cell creates a client that connects to this kernel and shows that it's alive.

In [7]:
client = BlockingKernelClient()
client.load_connection_file("/tmp/example_connection.json")
client.start_channels()
client.is_alive()

True

## Special tool

To make it easier to run jupyter kernels from juptyer notebook developed, a special tool has been developed. Use `utils.run_jupyter_kernel.run_kernel_in_process`. It takes arguments:

- `connection_file`: Path to the kernel connection file.
- `kernel_class`: Class of the kernel that determines the behaviour of the kernel.

---

The following cell runs kernel starts a new jupyter kernel.

In [2]:
from ipykernel.ipkernel import IPythonKernel
from utils.run_jupyter_kernel import run_kernel_in_process

run_kernel_in_process(
    connection_file="/tmp/run_kernel_example.json",
    kernel_class=IPythonKernel
)

NOTE: When using the `ipython kernel` entry point, Ctrl-C will not work.

To exit, you will have to explicitly quit this process, by either sending
"quit" from a client, or using Ctrl-\ in UNIX-like environments.

To read more about this, see https://github.com/ipython/ipython/issues/2049


To connect another client to this kernel, use:
    --existing /tmp/run_kernel_example.json


The following cell creates a client that proves that kernels works fine.

In [3]:
client = BlockingKernelClient()
client.load_connection_file("/tmp/run_kernel_example.json")
client.start_channels()
client.is_alive()

True